In [145]:
import requests
import json

from pathlib import Path
from pprint import pprint
from datetime import datetime

from openai import OpenAI
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [146]:
import os
from dotenv import load_dotenv

In [147]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Prompt
- load prompt
- load template with prompt

In [250]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()
print(prompt_continuous_effects)

prompt_template = PromptTemplate.from_template(prompt_continuous_effects)
#prompt_template = ChatPromptTemplate.from_template(prompt_continuous_effects)

"""
You act as a referee or judge in a game of Magic: the Gathering and have to judge a situation where two cards are played. {card1} and {card2} both have continuous effects.
Use the {rules_db} as context for your decision. If you make a decision for one rule, you have to stick with it.
You will get a {question} about two cards and give a judging regarding two contradicting continuous effects.
Make sure your judging, explanation und quoting of the rules don't contradict each other.

You will follow these steps:

1. Give the oracle texts from both {card1} and {card2}. Check the additional comments in the card informations.
1a. Determine the layer on which the effects of the cards are applied. Layers are defined in 613.1 in the {rules_db}.
1b. Decide if one card depends on the other like stated in rule 613.8a in the {rules_db}.
2. Decide which rule is fitting in the case: Timestamp (613.1 to 613.7) or Dependency (613.8) and quote the rule. Only one of the rule is the right answer to the

## RAG
= Retrieved Augmented Generation
- ChatGPT's knowlegde of the rules is from January 2022: for current state the rulebook needs to be retrieved and passed to the LLM
- this data can provide context for the specific questions
### Loader
- simple loader to load .txt data: complete rule book from Magic: The Gathering

In [152]:
#loader = TextLoader("./data/rules_shortened.txt")
loader = TextLoader("./data/rules.txt")
rules_doc = loader.load()

### Splitter
- splitting the text into smaller chunks
- this text splitter splits text recursively by characters: ["\n\n", "\n", " ", ""], so first paragraphs, then sentences, then words
- keeps semantically related pieces together as long as possible

In [153]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
# print(rules_splits[0])

### Vector Store
- for storage of the embeddings
- embedding for capture semantic meaning of the text: that's how context is given for later question
- OpenAI embedding for ChatGPT
- Chroma as open-source embedding database
- finally, construct a retriever out of vectorstore 

In [154]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever_rules = rules_db.as_retriever()

- create new LLM: temperature 0 for more consistent, non-creative and therefor comparable output

In [166]:
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- Get card information via scryfall.com and delete unnecessary data: 
- keep only: name, mana cost, converted mana cost, type, oracle text, colors, keywords

In [243]:
card1 = "Urborg, Tomb of Yawgmoth"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "rulings_uri", "set_search_uri", "scryfall_set_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
        
    # convert into json dict
    card1 = json.loads(json.dumps(data))
    
    # check for additional rulings via id
    card_id = card1.get('id')
    if card_id:
        response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                 headers={'Accept': 'application/json'})
    
    # add additional rules to card
        if response.status_code == 200:
            data_rulings = response.json()
            comments = [item.get('comment') for item in data_rulings.get('data', [])]
            card1['comments'] = comments
        
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)


{'id': '9e1a9e38-6ffc-490f-b0be-23ba4e8204c6', 'name': 'Urborg, Tomb of Yawgmoth', 'mana_cost': '', 'cmc': 0.0, 'type_line': 'Legendary Land', 'oracle_text': 'Each land is a Swamp in addition to its other land types.', 'colors': [], 'keywords': [], 'produced_mana': ['B'], 'comments': ["Urborg, Tomb of Yawgmoth isn't a Swamp while it's not on the battlefield.", "Land cards not on the battlefield aren't Swamps while Urborg is on the battlefield.", 'Urborg\'s ability causes each land on the battlefield to have the land type Swamp. Any land that\'s a Swamp has the ability "{T}: Add {B}." Nothing else changes about those lands, including their names, other subtypes, other abilities, and whether they\'re legendary, basic, or snow.', "If an effect such as that of Magus of the Moon causes Urborg to lose its abilities by setting it to a basic land type not in addition to its other types, it won't turn lands into Swamps, no matter in what order those effects started to apply."]}


In [246]:
card2 = "Blood Moon"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict
    card2 = json.loads(clean_json)
    
    # check for additional rulings via id
    card_id = card2.get('id')
    if card_id:
        response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                 headers={'Accept': 'application/json'})
    
    # add additional rules to card
        if response.status_code == 200:
            data_rulings = response.json()
            comments = [item.get('comment') for item in data_rulings.get('data', [])]
            card2['comments'] = comments
    
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

{'id': 'd072e9ca-aae7-45dc-8025-3ce590bae63f', 'name': 'Blood Moon', 'mana_cost': '{2}{R}', 'cmc': 3.0, 'type_line': 'Enchantment', 'oracle_text': 'Nonbasic lands are Mountains.', 'colors': ['R'], 'keywords': [], 'comments': ['If a nonbasic land has an ability that triggers “when” it enters the battlefield, it will lose that ability before it can trigger.', 'If a nonbasic land has an ability that causes it to enter the battlefield tapped, it will lose that ability before it can apply. The same is also true of any other abilities that modify how a land enters the battlefield or apply “as” a land enters the battlefield, such as the first ability of Cavern of Souls.', 'Nonbasic lands will lose any other land types and abilities they had. They will gain the land type Mountain and gain the ability “{T}: Add {R}.”', "This effect doesn't affect names or supertypes. It won't turn any land into a basic land or remove the legendary supertype from a legendary land, and the lands won't be named “M

In [251]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

- Use Chatmodel instead of LLM: more possible token in prompt
- Outputparser to parse message response from Chatmodel to str
- check testcases.md to compare expected outcome with ChatGPT's answer

In [252]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the final board state of two cards played with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Urborg, Tomb of Yawgmoth: Each land is a Swamp in addition to its other land types.
- Blood Moon: Nonbasic lands are Mountains.

1a. Layer:
The effects of Urborg, Tomb of Yawgmoth and Blood Moon are applied in layer 4, the type-changing effects layer.

1b. Dependency:
The effects of Urborg, Tomb of Yawgmoth and Blood Moon do not depend on each other.

2. Rule:
I would apply the Timestamp rule (613.7) in this case.

3. Explanation:
According to the Timestamp rule, the continuous effect of the card played first with the earlier timestamp takes precedence over the effect of the card played later with the later timestamp.

4. Judging:
In this scenario, Urborg, Tomb of Yawgmoth is played first and has the earlier timestamp. Therefore, its continuous effect of making each land a Swamp in addition to its other types would be applied. This means that all nonbasic lands would become Swamps, overriding the effect of Blood Moon that would turn nonbasic lands into Mountains. The

- saving the answers as .txt for comparison by date & time: 

In [156]:
current_datetime = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
file_name = current_datetime+".txt"

with open(f'./data/answers/{file_name}', 'w') as f:
    f.write(answer)

In [22]:
# cleanup
rules_db.delete_collection()

## Case 2a: Opalescence vs. Humility
- timestamp rule determines the outcome: *Humility* is played last and it's effect overrides *Opalescence*

In [185]:
card1 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': []}


In [186]:
card2 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Humility
{'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'keywords': []}


In [187]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [188]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle text of Opalescence: "Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value."
Oracle text of Humility: "All creatures lose all abilities and have base power and toughness 1/1."

2. I would choose the Timestamp rule (613.1 to 613.7) in this case. The Timestamp rule states that when two continuous effects are in conflict, the effect generated last "wins." In this scenario, Opalescence was played first and Humility was played second, so Humility's effect would be applied last.

3. I would apply the Timestamp rule in this case because it specifically addresses situations where conflicting continuous effects are in play and provides a clear resolution method.

4. Judging: The continuous effect of Humility would be applied in the game. This means that all creatures, including the enchantments turned into creatures by Opalescence, would lose all abilities and have base power and toughness of 1/

In [111]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

## Case 2b: Humility vs. Opalescence 
- because timestamp is important here, testing the case with changed order

In [206]:
card1 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'keywords': []}


In [207]:
card2 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Opalescence
{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': []}


In [208]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [209]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Humility: All creatures lose all abilities and have base power and toughness 1/1.
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.

1a. The effects of the cards are applied in different layers. Humility affects the abilities and power/toughness of creatures in layer 6 (ability adding or removing effects), while Opalescence affects the type-changing effects in layer 4 (type-changing effects).

1b. The text of the cards do not depend on each other as they are affecting different aspects of the game state.

2. The rule that is fitting in this case is the Timestamp rule (613.1 to 613.7).

3. I would apply the Timestamp rule because it determines the order in which continuous effects are applied based on when they entered the battlefield. In this case, Humility entered the battlefield first and therefore has the earlier timestamp.

4. Judging: The continuous effe

In [143]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

In [144]:
# cleanup
rules_db.delete_collection()